In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
home_sales_data.show()

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import round, avg


four_bedroom_houses = home_sales_data.filter(home_sales_data.bedrooms == 4)


average_prices = four_bedroom_houses.groupBy("year").agg(round(avg("price"), 2).alias("average_price")).orderBy("year")
average_prices.show()

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_rooms_houses = home_sales_data.filter((home_sales_data.bedrooms == 3) & (home_sales_data.bathrooms == 3))

average_price_by_built_date = three_rooms_houses.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price")).orderBy("date_built")
average_price_by_built_date.show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
filtered_houses = home_sales_data.filter(
    (home_sales_data.bedrooms == 3) &
    (home_sales_data.bathrooms == 3) &
    (home_sales_data.floors == 2) &
    (home_sales_data.sqft_lot >= 2000)
)


average_prices_filtered_houses = filtered_houses.groupBy("date_built") \
                                 .agg(round(avg("price"), 2).alias("average_price")) \
                                 .orderBy("date_built")


average_prices_filtered_houses.show()

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

filtered_houses = home_sales_data.filter(home_sales_data.price >= 350000)

average_prices_by_view = filtered_houses.groupBy("view") \
                                       .agg(round(avg("price"), 2).alias("average_price"))

# Show the result
average_prices_by_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

filtered_houses_cached = home_sales_data.filter(home_sales_data.price >= 350000)
average_prices_by_view_cached = filtered_houses_cached.groupBy("view") \
                                                       .agg(round(avg("price"), 2).alias("average_price"))
average_prices_by_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_data.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_p")

In [ ]:
# 11. Read the formatted parquet data.
p_home_sales=spark.read.parquet('home_sales_p')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView('p_home_sales_p')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

filtered_houses_p = p_home_sales.filter(p_home_sales.price >= 350000)
average_prices_by_view_p = filtered_houses_p.groupBy("view") \
                                                       .agg(round(avg("price"), 2).alias("average_price"))
average_prices_by_view_p.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")
